In [1]:
import mlagents


In [2]:
import random
import numpy as np
from mlagents_envs.environment import UnityEnvironment as UE

In [3]:
env = UE(file_name="4444_project",seed=1,side_channels=[])
env.reset()

In [4]:
print(list(env.behavior_specs))
behavior_name = list(env.behavior_specs)[0]
print(behavior_name)

['CarBehavior?team=0']
CarBehavior?team=0


In [5]:
print(type(env.behavior_specs[behavior_name]))
print()
print(env.behavior_specs[behavior_name])
spec = env.behavior_specs[behavior_name]

<class 'mlagents_envs.base_env.BehaviorSpec'>

BehaviorSpec(observation_specs=[ObservationSpec(shape=(8,), dimension_property=(<DimensionProperty.NONE: 1>,), observation_type=<ObservationType.DEFAULT: 0>, name='VectorSensor_size8')], action_spec=ActionSpec(continuous_size=1, discrete_branches=()))


In [6]:
print(type(spec.observation_specs))
print()
print(type(tuple(spec.observation_specs[0])))
print(spec.observation_specs[0])

<class 'list'>

<class 'tuple'>
ObservationSpec(shape=(8,), dimension_property=(<DimensionProperty.NONE: 1>,), observation_type=<ObservationType.DEFAULT: 0>, name='VectorSensor_size8')


In [7]:
print(type(spec.action_spec))
print()
print(spec.action_spec)

<class 'mlagents_envs.base_env.ActionSpec'>

Continuous: 1, Discrete: ()


In [8]:
decision_steps, terminal_steps = env.get_steps(behavior_name)

In [9]:
print(decision_steps)
print(list(decision_steps))

[0]


In [10]:
tracked_agent = -1
done = False
episode_rewards = 0

print(type(decision_steps.agent_id))
tracked_agent = decision_steps.agent_id[0]

<class 'numpy.ndarray'>


In [11]:
action = spec.action_spec.random_action(len(decision_steps))
print(action)
print(type(action.discrete))
print(action.discrete)
# action = action.continuous

<class 'numpy.ndarray'>
[]


In [12]:
env.set_actions(behavior_name, action)

In [13]:
for episode in range(10):
    env.reset()
    decision_steps, terminal_steps = env.get_steps(behavior_name)
    tracked_agent = -1 # -1 indicates not yet tracking
    done = False # For the tracked_agent
    episode_rewards = 0 # For the tracked_agent
    while not done:
        # Track the first agent we see if not tracking
        # Note : len(decision_steps) = [number of agents that requested a decision]
        if tracked_agent == -1 and len(decision_steps) >= 1:
            tracked_agent = decision_steps.agent_id[0]
        # Generate an action for all agents
        print(decision_steps.obs)
        action = spec.action_spec.random_action(len(decision_steps))
        #action = spec.action_spec.empty_action(len(decision_steps))
        #action.add_continuous(np.array([[0]]))
        # Set the actions
        env.set_actions(behavior_name, action)
        # Move the simulation forward
        env.step()
        # Get the new simulation results
        decision_steps, terminal_steps = env.get_steps(behavior_name)
        #print(decision_steps.obs)
        if tracked_agent in decision_steps: # The agent requested a decision
            episode_rewards += decision_steps[tracked_agent].reward
#             print(decision_steps[tracked_agent].reward)
        if tracked_agent in terminal_steps: # The agent terminated its episode
            episode_rewards += terminal_steps[tracked_agent].reward
#             print(terminal_steps[tracked_agent].reward)
            done = True
    print(f"Total rewards for episode {episode} is {episode_rewards}")

[array([[ 1.9999998e+01, -1.9620000e-01, -2.3841858e-06, -1.0000000e+00,
         8.4933500e+00,  6.1061568e+00,  1.2374710e+01,  8.2838812e+00]],
      dtype=float32)]
[array([[ 1.9999998e+01, -3.9240000e-01, -2.3841858e-06, -1.0000000e+00,
         8.5010386e+00,  6.0837030e+00,  1.1994226e+01,  8.5123911e+00]],
      dtype=float32)]
[array([[19.98996   , -0.5886    , -0.63367367, -1.        ,  8.499565  ,
         6.0795507 , 12.082872  ,  8.298254  ]], dtype=float32)]
[array([[19.994164  , -0.7848    , -0.48321247, -1.        ,  8.503579  ,
         6.0710363 , 11.720762  ,  8.473851  ]], dtype=float32)]
[array([[19.969347 , -0.981    , -1.1069489, -1.       ,  8.489261 ,
         6.075232 , 11.987339 ,  8.281328 ]], dtype=float32)]
[array([[19.99047   , -1.1772    , -0.61740875, -1.        ,  8.48822   ,
         6.0691476 , 11.839384  ,  8.396962  ]], dtype=float32)]
[array([[19.981285  , -1.3734    , -0.86505413, -1.        ,  8.481167  ,
         6.0682516 , 11.832667  ,  8.410

In [14]:
env.close()